![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [121]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [122]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like!

**Columns containing categories with only two factors must be stored as Booleans (bool).

Columns containing integers only must be stored as 32-bit integers (int32).

Columns containing floats must be stored as 16-bit floats (float16).**

In [123]:
dictionnaire={}
booleen=["job_change"]

for val in list(ds_jobs_transformed.columns):
    
    dictionnaire[val]=ds_jobs_transformed[val].nunique()
    if dictionnaire[val]==2:
        ds_jobs_transformed[val]=ds_jobs_transformed[val].astype(bool)
    if ds_jobs_transformed[val].dtype=="int64":
        ds_jobs_transformed[val]=ds_jobs_transformed[val].astype("int32")
    if ds_jobs_transformed[val].dtype=="float64":
        ds_jobs_transformed[val]=ds_jobs_transformed[val].astype("float16")
    



**Columns containing nominal categorical data must be stored as the category data type.**

**Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column.**

In [124]:
liste=["city","enrolled_university","education_level","major_discipline","experience","company_size","company_type","last_new_job","gender"]
for x in liste:
    ds_jobs_transformed[x]=ds_jobs_transformed[x].astype("object")
    ds_jobs_transformed[x]=ds_jobs_transformed[x].fillna("XXXX")
    ds_jobs_transformed[x]=ds_jobs_transformed[x].astype("category")
    ordre=list(ds_jobs_transformed[x].unique())
    categ=pd.CategoricalDtype(categories=ordre , ordered=True)
    ds_jobs_transformed[x]=ds_jobs_transformed[x].astype(categ)


In [125]:
for val in list(ds_jobs_transformed.columns):
    print("-------------------------------------------")
    print(val + "  " + str(ds_jobs_transformed[val].nunique()))
    print('***********************************************')
    print(ds_jobs_transformed[val].unique())

-------------------------------------------
student_id  19158
***********************************************
[ 8949 29725 11561 ... 24576  5756 23834]
-------------------------------------------
city  123
***********************************************
['city_103', 'city_40', 'city_21', 'city_115', 'city_162', ..., 'city_121', 'city_129', 'city_8', 'city_31', 'city_171']
Length: 123
Categories (123, object): ['city_103' < 'city_40' < 'city_21' < 'city_115' ... 'city_129' < 'city_8' < 'city_31' < 'city_171']
-------------------------------------------
city_development_index  93
***********************************************
[0.92  0.776 0.624 0.789 0.767 0.764 0.762 0.913 0.926 0.827 0.843 0.804
 0.855 0.887 0.91  0.884 0.924 0.666 0.558 0.923 0.794 0.754 0.939 0.55
 0.865 0.698 0.893 0.796 0.866 0.682 0.802 0.579 0.878 0.897 0.949 0.925
 0.896 0.836 0.693 0.769 0.775 0.903 0.555 0.727 0.64  0.516 0.743 0.899
 0.915 0.689 0.895 0.89  0.847 0.527 0.766 0.738 0.647 0.795 0.74  0.701
 0.

**The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies.**

In [126]:
liste_exp=['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20','>20','XXXX']
liste_com=['<10','10-49','50-99','100-499','500-999','1000-4999', '5000-9999','10000+','XXXX']

categ=pd.CategoricalDtype(categories=ordre , ordered=True)
ds_jobs_transformed["experience"]=ds_jobs_transformed["experience"].astype(categ)

categ=pd.CategoricalDtype(categories=ordre , ordered=True)
ds_jobs_transformed["company_size"]=ds_jobs_transformed["company_size"].astype(categ)

#ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed['experience'] >= '10') & (ds_jobs_transformed['company_size'] >= '1000-4999')]

print(ds_jobs_transformed.shape)

#cond1=(ds_jobs_transformed["experience"]>='10')
#cond2=(ds_jobs_transformed["company_size"]>='1000-4999')

#ds_jobs_transformed=ds_jobs_transformed[(ds_jobs_transformed["experience"]>='10') & (ds_jobs_transformed["company_size"]>='1000-4999')]

print(ds_jobs_transformed[["experience","company_size"]].head())


(19158, 14)
  experience company_size
0        NaN         XXXX
1        NaN          NaN
2        NaN         XXXX
3        NaN         XXXX
4        NaN          NaN


In [127]:
ds_jobs_transformed = ds_jobs.copy()

# Create a dictionary of columns containing ordered categorical data
ordered_cats = {
    'enrolled_university': ['no_enrollment', 'Part time course', 'Full time course'],
    'education_level': ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'],
    'experience': ['<1'] + list(map(str, range(1, 21))) + ['>20'],
    'company_size': ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'],
    'last_new_job': ['never', '1', '2', '3', '4', '>4']
}

# Create a mapping dictionary of columns containing two-factor categories to convert to Booleans
two_factor_cats = {
    'relevant_experience': {'No relevant experience': False, 'Has relevant experience': True},
    'job_change': {0.0: False, 1.0: True}
}

# Loop through DataFrame columns to efficiently change data types
for col in ds_jobs_transformed:
    
    # Convert two-factor categories to bool
    if col in ['relevant_experience', 'job_change']:
        ds_jobs_transformed[col] = ds_jobs_transformed[col].map(two_factor_cats[col])
    
    # Convert integer columns to int32
    elif col in ['student_id', 'training_hours']:
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('int32')
    
    # Convert float columns to float16
    elif col == 'city_development_index':
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('float16')
    
    # Convert columns containing ordered categorical data to ordered categories using dict
    elif col in ordered_cats.keys():
        category = pd.CategoricalDtype(ordered_cats[col], ordered=True)
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype(category)
    
    # Convert remaining columns to standard categories
    else:
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('category')
        
# Filter students with 10 or more years experience at companies with at least 1000 employees
ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed['experience'] >= '10') & (ds_jobs_transformed['company_size'] >= '1000-4999')]